# Memory

In [ ]:
import os
import warnings
from IPython.display import Markdown
from setenv import APIKeyManager

# 忽略警告
warnings.filterwarnings('ignore')
# 设置API密钥
key_manager = APIKeyManager().setup_api_key(env_names=["OPENAI_API_KEY", "OPENAI_API_BASE"])

# !pip install chromadb
# !pip install langchain-deepseek

# 查看Python版本
!python -V
# 查看安装的库
!pip list | grep -E 'langchain|openai|llm|tiktoken|chromadb|cryptography|duck'

## 1. 记忆类型

### 1.1 BufferMemory 

ConversationBufferMemory 是最简单的记忆类型，它将所有对话历史存储在列表中<br/>
memory_key 参数指定在链的输入中使用哪个键来存储对话历史<br/>
return_messages=True 表示返回消息对象列表而不是字符串<br/>
这种记忆类型适合短对话，但对于长对话可能会占用过多内存

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

# 创建LLM
llm = ChatOpenAI(temperature=0.7, model="deepseek-chat")

# 创建缓冲记忆
memory = ConversationBufferMemory(
    memory_key="chat_history",  # 指定记忆在链输入中的键名
    return_messages=True        # 返回消息对象而非字符串
)

# 创建提示模板
prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template="聊天历史：{chat_history}\n用户输入：{input}\n回答："
)

# 在链中使用
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.invoke({"input": "你好，我是小明。"})

In [ ]:
chain.invoke({"input": "我是Rookie"})

### 1.2 VectorStoreMemory
VectorStoreMemory 使用向量存储来保存对话历史，适合长对话和语义搜索<br/>
它可以根据当前查询检索相关的历史对话，而不是简单地返回所有历史<br/>
这种记忆类型特别适合需要从大量历史对话中检索相关信息的场景<br/>
下面是一个简单的VectorStoreMemory示例

In [ ]:
import os  
import requests  
from langchain.embeddings.base import Embeddings  
from langchain_community.vectorstores import FAISS  
import tempfile  
from langchain.memory import VectorStoreRetrieverMemory  
from langchain_openai import OpenAIEmbeddings

# 自定义DeepSeek嵌入类  
class DeepSeekEmbeddings(Embeddings):  
    def __init__(self, api_key):  
        self.api_key = api_key  
    
    def embed_documents(self, texts):  
        headers = {  
            'Authorization': f'Bearer {self.api_key}',  
            'Content-Type': 'application/json'  
        }  
        response = requests.post(  
            'https://api.deepseek.com/embeddings',  
            json={'input': texts, 'model': 'deepseek-embed'},  
            headers=headers  
        )  
        response.raise_for_status()  
        return [item['embedding'] for item in response.json()['data']]
    
    def embed_query(self, text):  
        return self.embed_documents([text])[0]

embeddings = DeepSeekEmbeddings(api_key=os.environ["OPENAI_API_KEY"])

# 创建临时目录来存储FAISS索引  
persist_directory = tempfile.mkdtemp()  

# 使用FAISS创建向量存储  
vectorstore = FAISS.from_texts(  
    ["初始化向量存储"],   
    embedding=embeddings  
)  

# 创建记忆组件  
memory = VectorStoreRetrieverMemory(  
    retriever=vectorstore.as_retriever(),  
    memory_key="chat_history"  
)

In [ ]:
# 向记忆中添加一些示例对话
# 注意：由于出现NotFoundError: Error code: 404错误，
# 可能是向量存储配置有问题，我们需要重新初始化向量存储
# 确保向量存储正确创建并持久化到磁盘

# 创建一个本地持久化的向量存储


# 现在尝试保存上下文
memory.save_context(
    {"input": "我最喜欢的颜色是蓝色"},
    {"output": "蓝色是一个很棒的颜色！它让人联想到海洋和天空。"}
)

memory.save_context(
    {"input": "我喜欢在周末去爬山"},
    {"output": "爬山是一项很好的活动，既能锻炼身体又能亲近自然。"}
)

memory.save_context(
    {"input": "我养了一只叫小花的猫"},
    {"output": "小花是个可爱的名字！猫咪是很好的宠物伴侣。"}
)


prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好的助手，能够记住用户之前提到的信息。"),
    ("human", "{input}"),
    ("human", "相关的对话历史：\n{chat_history}")
])

# 创建链
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7, model="deepseek-chat")
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

# 测试向量存储记忆
response = chain.invoke({"input": "我之前说过我喜欢什么颜色？"})